In [ ]:
%pip -q install --upgrade ultralytics==8.3.199 pillow opencv-python tqdm

from google.colab import drive
drive.mount('/content/drive')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 68.5 MB/s eta 0:00:00
Mounted at /content/drive


In [ ]:
from pathlib import Path
import os, shutil, glob

# Proje kökü – senin verdiğin yapı
BASE = Path("/content/drive/MyDrive/censorly-dataset/Alcohol/Alcohol-True/Edited-Alcohol-3")
IMAGES = BASE/"images"            # dokunmuyoruz
LABELS = BASE/"labels"            # BINARY (hepsi 0) — artık eğitim bu klasörden
ORIG   = BASE/"original_labels"   # çok sınıflı yedek (dokunmuyoruz)

assert IMAGES.exists(), "images klasörü bulunamadı!"
assert LABELS.exists(), "labels (binary) klasörü bulunamadı! Önce dönüştürme adımını tamamla."
print("Klasörler OK ✅")

# 'val' veya 'validation' hangisi varsa onu kullan
use_val = "val" if (IMAGES/"val").exists() else ("validation" if (IMAGES/"validation").exists() else None)
assert use_val is not None, "images altında 'val' ya da 'validation' klasörü bulunamadı!"

from pathlib import Path
DATA_YAML = BASE/"data.yaml"
DATA_YAML.write_text(f"""# binary alcohol detector (labels = binary)
path: {BASE.as_posix()}
train: images/train
val: images/{use_val}
test: images/test

names:
  0: alcohol
""")
print("data.yaml ->", DATA_YAML)
print(DATA_YAML.read_text())

Klasörler OK ✅
data.yaml -> /content/drive/MyDrive/censorly-dataset/Alcohol/Alcohol-True/Edited-Alcohol-3/data.yaml
# binary alcohol detector (labels = binary)
path: /content/drive/MyDrive/censorly-dataset/Alcohol/Alcohol-True/Edited-Alcohol-3
train: images/train
val: images/validation
test: images/test

names:
  0: alcohol



In [ ]:
# Test split (val değil)
test_metrics = model.val(
    data=str(DATA_YAML),
    imgsz=IMGSZ,
    split='test',
    project=str(RUN_DIR),
    name=f"{run_name}_test",
    exist_ok=True
)
test_metrics

In [ ]:
# ==========================================
# YOLOv11/YOLOv8 DETECTION — Alcohol True vs False (oranı koru, 1:1 dayatma yok)
# True:  /MyDrive/censorly-dataset/Alcohol/Alcohol-True/Edited-Alcohol-3/{images,labels}/{train,validation,test}
# False: /MyDrive/censorly-dataset/Alcohol/Alcohol-False/Alcohol-False/{images,labels}/{train,validation,test}
# ==========================================
%pip -q install --upgrade "ultralytics>=8.3.0" pillow opencv-python tqdm

import os, shutil, time, random
from datetime import datetime
from pathlib import Path
from ultralytics import YOLO

# ---------- Kullanıcı ayarları ----------
IMGSZ        = 960
EPOCHS       = 70
BATCH        = -1       # AutoBatch
WORKERS      = 2        # Drive yavaşsa 0 yap
SEED         = 42
RUN_NAME     = f"yolo11s_true_false_detect_{IMGSZ}_symlink"
STAGE_MODE   = "symlink"   # "symlink" önerilir; "copy" istersen değiştir

# ---------- Drive & yollar ----------
try:
    from google.colab import drive
    if not os.path.isdir('/content/drive/MyDrive'):
        print("[MOUNT] Drive bağlanıyor...")
        drive.mount('/content/drive')
except Exception:
    pass

DRIVE      = Path('/content/drive/MyDrive')
DATA_ROOT  = DRIVE/'censorly-dataset'/'Alcohol'

# ✅ True klasörü yeni dizine göre
TRUE_DIR   = DATA_ROOT/'Alcohol-True'/'Edited-Alcohol-3'   # images/labels/{train,validation,test}
# ✅ False aynı kalıyor
FALSE_DIR  = DATA_ROOT/'Alcohol-False'/'Alcohol-False'     # images/labels/{train,validation,test}

# Tüm run’ların, modellerin ve değerlendirme çıktılarının Drive’a yazılacağı ana klasör
RUN_DIR       = DRIVE/'censorly-training'/'alcohol'
PROJECT_DIR   = RUN_DIR/'runs_detect'  # Ultralytics 'project' paramı -> hepsi Drive’a
RUN_DIR.mkdir(parents=True, exist_ok=True)
PROJECT_DIR.mkdir(parents=True, exist_ok=True)

# Staging (eğitim okuma yolu; yerel/kolab diski)
STAGE_ROOT = Path('/content/datasets/alcohol_mix')
for s in ['train','validation','test']:
    for sub in ['images','labels']:
        (STAGE_ROOT/sub/s).mkdir(parents=True, exist_ok=True)

# ---------- Yardımcılar ----------
IMG_EXTS = {'.jpg','.jpeg','.png','.bmp','.webp','.tif','.tiff','.jfif','.heic','.heif'}
def is_img(p: Path): return p.is_file() and p.suffix.lower() in IMG_EXTS

def layout_ok(base: Path):
    # images/{train,validation,test} ve labels/{train,validation,test} var mı?
    need = [(base/'images'/'train'), (base/'images'/'validation'), (base/'images'/'test'),
            (base/'labels'/'train'), (base/'labels'/'validation'), (base/'labels'/'test')]
    return all(p.exists() for p in need)

def scan_split(base: Path, split: str, must_have_label: bool):
    """base: sınıf kökü (Edited-Alcohol-3 ya da Alcohol-False), split: train/validation/test"""
    imgd = base/'images'/split
    lbld = base/'labels'/split
    items = []
    if not imgd.exists():
        return items
    for img in imgd.rglob('*'):
        if not is_img(img):
            continue
        lbl = lbld/(img.stem + '.txt')
        if must_have_label:
            if lbl.exists():
                items.append((img, lbl))
        else:
            # Negatif için boş label yoksa oluştur
            if not lbl.exists():
                lbl.parent.mkdir(parents=True, exist_ok=True)
                lbl.write_text('', encoding='utf-8')
            items.append((img, lbl))
    return items

def safe_symlink(src: Path, dst: Path):
    if dst.exists() or dst.is_symlink():
        return
    try:
        os.symlink(src.as_posix(), dst.as_posix())
    except FileExistsError:
        pass

def stage_pair(src_img: Path, src_lbl: Path, dst_img: Path, dst_lbl: Path, mode="symlink"):
    if mode == "symlink":
        safe_symlink(src_img, dst_img)
        if src_lbl.exists(): safe_symlink(src_lbl, dst_lbl)
        else: dst_lbl.write_text('', encoding='utf-8')
    else:
        if not dst_img.exists():
            shutil.copy2(src_img, dst_img)
        if src_lbl.exists():
            if not dst_lbl.exists():
                shutil.copy2(src_lbl, dst_lbl)
        else:
            dst_lbl.write_text('', encoding='utf-8')

# ---------- Kontroller ----------
print("\n[CHECK] TRUE:", TRUE_DIR)
if not TRUE_DIR.exists() or not layout_ok(TRUE_DIR):
    raise FileNotFoundError(
        f"[ERR] True layout hatalı: {TRUE_DIR}\n"
        "Beklenen: .../Alcohol-True/Edited-Alcohol-3/{images,labels}/{train,validation,test}"
    )
print("[OK] True layout doğru.")

print("[CHECK] FALSE:", FALSE_DIR)
if not FALSE_DIR.exists() or not layout_ok(FALSE_DIR):
    raise FileNotFoundError(
        f"[ERR] False layout hatalı: {FALSE_DIR}\n"
        "Beklenen: .../Alcohol-False/Alcohol-False/{images,labels}/{train,validation,test}"
    )
print("[OK] False layout doğru.")

# ---------- Sayımlar ve staging ----------
random.seed(SEED)
splits = ['train','validation','test']
for split in splits:
    pos = scan_split(TRUE_DIR,  split, must_have_label=True)
    neg = scan_split(FALSE_DIR, split, must_have_label=False)  # boş txt'ler tamamlanır
    print(f"[COUNT {split:<10}] pos={len(pos):4d} | neg={len(neg):4d}")

    print(f"\n[STAGE {split}] Tüm pozitif + mevcut tüm negatif taşınıyor (mode={STAGE_MODE})")
    img_out = STAGE_ROOT/'images'/split
    lbl_out = STAGE_ROOT/'labels'/split

    # Pozitifleri taşı (etiketli)
    cp_p = 0
    for img, lbl in pos:
        stem = f"p__{img.stem}"
        dst_img = img_out/(stem + img.suffix.lower())
        dst_lbl = lbl_out/(stem + '.txt')
        stage_pair(img, lbl, dst_img, dst_lbl, STAGE_MODE)
        cp_p += 1

    # Negatifleri taşı (boş etiket)
    cp_n = 0
    for img, lbl in neg:
        stem = f"n__{img.stem}"
        dst_img = img_out/(stem + img.suffix.lower())
        dst_lbl = lbl_out/(stem + '.txt')
        stage_pair(img, lbl, dst_img, dst_lbl, STAGE_MODE)
        cp_n += 1

    print(f"[DONE {split}] +pos={cp_p}  +neg={cp_n}")

# ---------- data.yaml ----------
DATA_YAML = RUN_DIR/'data_true_false_detect.yaml'
DATA_YAML.write_text(
    "\n".join([
        "train:",
        f"  - {str((STAGE_ROOT/'images'/'train').resolve())}",
        "val:",
        f"  - {str((STAGE_ROOT/'images'/'validation').resolve())}",
        "test:",
        f"  - {str((STAGE_ROOT/'images'/'test').resolve())}",
        "names:",
        "  0: alcohol",
    ]) + "\n",
    encoding='utf-8'
)
print("\n[YAML]", DATA_YAML)

# ---------- Ağırlık seçimi ----------
candidate_weights = ['yolo11s.pt', 'yolov8s.pt']
WEIGHTS_IN = None
for w in candidate_weights:
    try:
        _ = YOLO(w)  # erişilebilir mi?
        WEIGHTS_IN = w
        break
    except Exception:
        WEIGHTS_IN = None
if WEIGHTS_IN is None:
    WEIGHTS_IN = 'yolov8s.pt'
print("[MODEL] start_weights =", WEIGHTS_IN)

# ---------- Eğitim ----------
print("\n[TRAIN] başlıyor… (task=detect, single_cls=True)")
model = YOLO(WEIGHTS_IN)
results = model.train(
    data=str(DATA_YAML),
    epochs=EPOCHS,
    imgsz=IMGSZ,
    batch=BATCH,
    workers=WORKERS,
    project=str(PROJECT_DIR),
    name=RUN_NAME,
    exist_ok=True,
    device=0,
    patience=20,
    cos_lr=True,
    optimizer='AdamW',
    lr0=0.0015,
    seed=SEED,
    deterministic=True,
    single_cls=True,   # tek sınıf (alcohol)
    close_mosaic=10,
    cache=False        # Drive yavaşsa STAGE_MODE='copy' + cache=True yapabilirsin
)

# ---------- Eğitim klasörünü Drive’da sakla (tümüyle) ----------
save_dir  = Path(getattr(getattr(model,'trainer',None),'save_dir', PROJECT_DIR/ RUN_NAME))
print("[SAVE_DIR]", save_dir)

# timestamp’lı bir kopya (istersen direkt save_dir Drive’da zaten; yine de arşivlemek iyi olur)
stamp = datetime.now().strftime("%Y%m%d_%H%M%S")
archive_dir = RUN_DIR/'archives'/f"{RUN_NAME}_{stamp}"
archive_dir.parent.mkdir(parents=True, exist_ok=True)
if archive_dir.exists():
    shutil.rmtree(archive_dir)
shutil.copytree(save_dir, archive_dir)
print(f"[ARCHIVE] Eğitim klasörü arşivlendi -> {archive_dir}")

# Ek olarak best/last ağırlıklarını da düz kopya olarak atalım
weights   = save_dir/'weights'
best_src  = weights/'best.pt'
last_src  = weights/'last.pt'
best_dst  = RUN_DIR/f'best_detect_{IMGSZ}_symlink.pt'
last_dst  = RUN_DIR/f'last_detect_{IMGSZ}_symlink.pt'
if best_src.exists():
    shutil.copy2(best_src, best_dst); print(f"[SAVE] {best_src}  ->  {best_dst}")
if last_src.exists():
    shutil.copy2(last_src, last_dst); print(f"[SAVE] {last_src}  ->  {last_dst}")

# ---------- TEST split değerlendirmesi (val() ile) ----------
print("\n[VAL/TEST] Test split üzerinde değerlendirme başlıyor…")
val_out_name = f"{RUN_NAME}_TEST_EVAL"
val_results = model.val(
    data=str(DATA_YAML),
    imgsz=IMGSZ,
    split='test',           # test split
    save=True,              # PR/ROC/CM vs. plotları kaydet
    plots=True,
    project=str(PROJECT_DIR),
    name=val_out_name,
    conf=0.25,
    iou=0.7
)
print("[VAL DONE] Çıktılar ->", PROJECT_DIR/val_out_name)

# ---------- Test görsellerinden örnek tahminler ----------
print("\n[PREDICT] Test görsellerinden örnek tahminler kaydediliyor…")
TEST_IMG_DIR = (STAGE_ROOT/'images'/'test')
pred_name = f"{RUN_NAME}_pred_examples"
pred_results = model.predict(
    source=str(TEST_IMG_DIR),
    imgsz=IMGSZ,
    conf=0.25,
    iou=0.7,
    save=True,              # çizilmiş görselleri kaydet
    save_txt=True,          # txt çıkışı
    save_conf=True,         # txt’de confidence yaz
    project=str(PROJECT_DIR),
    name=pred_name,
    max_det=50,
    verbose=False
)
print("[PRED DONE] Örnekler ->", PROJECT_DIR/pred_name)

print("\n[FINISHED] Eğitim + Test değerlendirme + Örnek pred. tamam.")
print("Eğitim klasörü:", save_dir)
print("Arşiv klasörü  :", archive_dir)
print("Test eval klas.:", PROJECT_DIR/val_out_name)
print("Pred örnek klas:", PROJECT_DIR/pred_name)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 54.7 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
[MOUNT] Drive bağlanıyor...
Mounted at /content/drive

[CHECK] TRUE: /content/drive/MyDrive/censorly-dataset/Alcohol/Alcohol-True/Edited-Alcohol-3
[OK] True layout doğru.
[CHECK] FALSE: /content/drive/MyDrive/censorly-dataset/Alcohol/Alcohol-False/Alcohol-False
[OK] False layout doğru.
[COUNT train     ] pos=1782 | neg= 342

[STAGE train] Tüm pozitif + mevcut tüm negatif taşınıyor (mode=symlink)
[DONE train] +pos=1782  +neg=342
[COUNT validation] pos= 197 | neg= 123

[STAGE validation] Tüm pozitif + mevcut tüm negatif taşınıyor (mode=symlink)
[DONE validation] +pos=197  +neg=123
[COUNT test      ] pos= 19

In [ ]:
import os, glob, csv, json, math
from pathlib import Path
from datetime import datetime

import cv2
import numpy as np
import torch
from ultralytics import YOLO

# ====================== KULLANICI AYARLARI ======================
MODEL_WEIGHTS = "/content/drive/MyDrive/censorly-training/alcohol/finetune_with_all_neg2/weights/best.pt"

IMG_DIR = "/content/drive/MyDrive/censorly-dataset/Alcohol/Alcohol-True/Edited-Alcohol-3/images/test"
LBL_DIR = "/content/drive/MyDrive/censorly-dataset/Alcohol/Alcohol-True/Edited-Alcohol-3/labels/test"

SAVE_VIS = True  # kutulu görselleri kaydet
CONF_INFER = 0.30  # inference için düşük eşik
IMGSZ = 960

# Değerlendirme eşikleri
IOU_THR = 0.50              # standart IoU metriği için
CONTAIN_THR = 0.80          # pred'in en az bu oranla GT içinde olması (inter/pred_area)
MIN_PRED_CONF_FOR_EVAL = 0.10  # değerlendirmeye dahil edilmesi için min skor

# Çıktı klasörü
ts = datetime.now().strftime("%Y%m%d_%H%M%S")
OUT_DIR = Path(f"/content/drive/MyDrive/censorly-training/alcohol/contain_eval_{ts}")
OUT_DIR.mkdir(parents=True, exist_ok=True)
VIS_DIR = OUT_DIR / "viz"
if SAVE_VIS:
    VIS_DIR.mkdir(parents=True, exist_ok=True)

# ===============================================================


def yolo_label_to_xyxy(lbl, w, h):
    # YOLO txt: cls cx cy bw bh (hepsi [0,1])
    c, cx, cy, bw, bh = lbl
    x1 = (cx - bw / 2.0) * w
    y1 = (cy - bh / 2.0) * h
    x2 = (cx + bw / 2.0) * w
    y2 = (cy + bh / 2.0) * h
    return int(c), np.array([x1, y1, x2, y2], dtype=np.float32)

def iou_xyxy(a, b):
    x1 = max(a[0], b[0]); y1 = max(a[1], b[1])
    x2 = min(a[2], b[2]); y2 = min(a[3], b[3])
    iw = max(0.0, x2 - x1); ih = max(0.0, y2 - y1)
    inter = iw * ih
    area_a = max(0.0, (a[2]-a[0]) * (a[3]-a[1]))
    area_b = max(0.0, (b[2]-b[0]) * (b[3]-b[1]))
    union = area_a + area_b - inter + 1e-9
    return inter / union

def inter_over_pred_area(pred, gt):
    x1 = max(pred[0], gt[0]); y1 = max(pred[1], gt[1])
    x2 = min(pred[2], gt[2]); y2 = min(pred[3], gt[3])
    iw = max(0.0, x2 - x1); ih = max(0.0, y2 - y1)
    inter = iw * ih
    p_area = max(1e-9, (pred[2]-pred[0]) * (pred[3]-pred[1]))
    return inter / p_area

def greedy_iou_match(gts, preds, thr=0.5):
    """
    gts: [G,4], preds: [P,4] (skorları -> sıralama için ayrıca ver)
    1-e-1 TP sayımı için greedy IoU eşleştirme.
    """
    G = len(gts); P = len(preds)
    used_g = set(); used_p = set()
    tps = 0
    for pi in range(P):
        best_iou, best_g = -1.0, -1
        for gi in range(G):
            if gi in used_g:
                continue
            iou = iou_xyxy(preds[pi], gts[gi])
            if iou >= thr and iou > best_iou:
                best_iou, best_g = iou, gi
        if best_g >= 0:
            used_g.add(best_g); used_p.add(pi); tps += 1
    fps = P - tps
    fns = G - len(used_g)
    return tps, fps, fns

def containment_count(gts, preds, thr=0.8):
    """
    Bir GT içine giren tüm pred'ler TP.
    TP: inter/pred_area >= thr sağlayan tüm pred'ler.
    FP: diğer tüm pred'ler.
    FN: hiçbir pred tarafından 'yeterince kapsanmayan' GT'ler.
    """
    P = len(preds); G = len(gts)
    if P == 0 and G == 0:
        return 0, 0, 0
    if P == 0:
        return 0, 0, G
    if G == 0:
        return 0, P, 0

    tp_flags = np.zeros(P, dtype=bool)
    gt_hit = np.zeros(G, dtype=bool)

    for pi in range(P):
        for gi in range(G):
            r = inter_over_pred_area(preds[pi], gts[gi])
            if r >= thr:
                tp_flags[pi] = True
                gt_hit[gi] = True  # en az bir pred bu GT'yi kapsadı
    TP = int(tp_flags.sum())
    FP = int(P - TP)
    FN = int(G - gt_hit.sum())
    return TP, FP, FN

def draw(img, gts, preds, title=""):
    out = img.copy()
    font = cv2.FONT_HERSHEY_SIMPLEX
    # GT: YEŞİL
    for b in gts:
        x1,y1,x2,y2 = b.astype(int)
        cv2.rectangle(out, (x1,y1), (x2,y2), (60,180,60), 2)
    # Pred: MAVİ
    for b in preds:
        x1,y1,x2,y2 = b.astype(int)
        cv2.rectangle(out, (x1,y1), (x2,y2), (255,120,0), 2)
    if title:
        cv2.putText(out, title, (10,25), font, 0.7, (0,0,0), 3, cv2.LINE_AA)
        cv2.putText(out, title, (10,25), font, 0.7, (255,255,255), 1, cv2.LINE_AA)
    return out

# --- Model ---
assert Path(MODEL_WEIGHTS).exists(), f"Model yok: {MODEL_WEIGHTS}"
model = YOLO(MODEL_WEIGHTS)

# --- Görseller ---
img_exts = ("*.jpg","*.jpeg","*.png","*.bmp","*.webp")
img_paths = []
for ext in img_exts:
    img_paths += glob.glob(str(Path(IMG_DIR)/ext))
img_paths = sorted(img_paths)
assert len(img_paths) > 0, f"Görsel bulunamadı: {IMG_DIR}"

# --- CSV hazırlık ---
per_img_csv = OUT_DIR / "per_image_metrics.csv"
detail_csv = OUT_DIR / "boxes_detail.csv"

with open(per_img_csv, "w", newline="") as f1, open(detail_csv, "w", newline="") as f2:
    w1 = csv.writer(f1)
    w2 = csv.writer(f2)
    w1.writerow([
        "image","gt_count","pred_count",
        "TP_iou","FP_iou","FN_iou",
        "TP_cont","FP_cont","FN_cont"
    ])
    w2.writerow([
        "image","kind","x1","y1","x2","y2","conf"
    ])

    # --- Toplamlar ---
    TP_iou=FP_iou=FN_iou=0
    TP_cont=FP_cont=FN_cont=0

    for res in model.predict(source=img_paths, conf=CONF_INFER, imgsz=IMGSZ, save=False, stream=True, verbose=False):
        im_path = Path(res.path)
        img = cv2.imread(str(im_path)); H,W = img.shape[:2]

        # GT oku
        gt_file = Path(LBL_DIR) / (im_path.stem + ".txt")
        gts = []
        if gt_file.exists():
            with open(gt_file, "r") as f:
                for line in f:
                    parts = line.strip().split()
                    if len(parts) != 5:
                        continue
                    c,cx,cy,bw,bh = map(float, parts)
                    _, box = yolo_label_to_xyxy([c,cx,cy,bw,bh], W, H)
                    gts.append(box)
        gts = np.array(gts, dtype=np.float32)

        # Pred'ler
        preds = []
        if res.boxes is not None and len(res.boxes) > 0:
            xyxy = res.boxes.xyxy.detach().cpu().numpy()
            conf = res.boxes.conf.detach().cpu().numpy()
            # değerlendirmeye alınacak skorlar
            mask = conf >= MIN_PRED_CONF_FOR_EVAL
            xyxy = xyxy[mask]; conf = conf[mask]
            preds = xyxy

            # detay CSV: pred
            for i in range(len(xyxy)):
                x1,y1,x2,y2 = xyxy[i].tolist()
                w2.writerow([str(im_path),"pred",x1,y1,x2,y2,float(conf[i])])
        else:
            preds = np.zeros((0,4), dtype=np.float32)

        # detay CSV: gt
        for box in gts:
            x1,y1,x2,y2 = box.tolist()
            w2.writerow([str(im_path),"gt",x1,y1,x2,y2,""])

        # --- METRİKLER ---
        # IoU: skor sıralı greedy 1-e-1 (skora göre sırala)
        if preds.shape[0] > 0 and res.boxes is not None:
            conf_all = res.boxes.conf.detach().cpu().numpy()
            if len(conf_all) == preds.shape[0]:
                order = np.argsort(-conf_all)
                preds_iou = preds[order]
            else:
                preds_iou = preds
        else:
            preds_iou = preds

        ti,fi,ni = greedy_iou_match(gts, preds_iou, thr=IOU_THR)
        tc,fc,nc = containment_count(gts, preds, thr=CONTAIN_THR)

        TP_iou += ti; FP_iou += fi; FN_iou += ni
        TP_cont += tc; FP_cont += fc; FN_cont += nc

        # per-image yaz
        w1.writerow([str(im_path), len(gts), len(preds), ti, fi, ni, tc, fc, nc])

        # görsel kaydet
        if SAVE_VIS:
            title = f"IOU(TP:{ti}/FP:{fi}/FN:{ni})  |  CONT(TP:{tc}/FP:{fc}/FN:{nc})"
            vis = draw(img, gts, preds, title=title)
            cv2.imwrite(str(VIS_DIR / f"{im_path.stem}.jpg"), vis)

# --- Özet rapor ---
def prf(tp, fp, fn):
    prec = tp / (tp+fp) if (tp+fp)>0 else 0.0
    rec  = tp / (tp+fn) if (tp+fn)>0 else 0.0
    f1   = (2*prec*rec/(prec+rec)) if (prec+rec)>0 else 0.0
    return round(prec,4), round(rec,4), round(f1,4)

prec_i, rec_i, f1_i = prf(TP_iou, FP_iou, FN_iou)
prec_c, rec_c, f1_c = prf(TP_cont, FP_cont, FN_cont)

summary = {
    "model": MODEL_WEIGHTS,
    "img_dir": IMG_DIR,
    "lbl_dir": LBL_DIR,
    "images_evaluated": len(img_paths),
    "iou_thr": IOU_THR,
    "contain_thr": CONTAIN_THR,
    "min_pred_conf_eval": MIN_PRED_CONF_FOR_EVAL,
    "iou_TP": TP_iou, "iou_FP": FP_iou, "iou_FN": FN_iou,
    "iou_precision": prec_i, "iou_recall": rec_i, "iou_f1": f1_i,
    "contain_TP": TP_cont, "contain_FP": FP_cont, "contain_FN": FN_cont,
    "contain_precision": prec_c, "contain_recall": rec_c, "contain_f1": f1_c,
    "per_image_csv": str(per_img_csv),
    "boxes_detail_csv": str(detail_csv),
    "viz_dir": str(VIS_DIR) if SAVE_VIS else None,
}

with open(OUT_DIR/"summary.json", "w") as f:
    json.dump(summary, f, indent=2)

print("\n=== SUMMARY ===")
for k,v in summary.items():
    print(f"{k}: {v}")
print(f"\nOutputs saved to: {OUT_DIR}")


=== SUMMARY ===
model: /content/drive/MyDrive/censorly-training/alcohol/finetune_with_all_neg2/weights/best.pt
img_dir: /content/drive/MyDrive/censorly-dataset/Alcohol/Alcohol-True/Edited-Alcohol-3/images/test
lbl_dir: /content/drive/MyDrive/censorly-dataset/Alcohol/Alcohol-True/Edited-Alcohol-3/labels/test
images_evaluated: 199
iou_thr: 0.5
contain_thr: 0.8
min_pred_conf_eval: 0.1
iou_TP: 271
iou_FP: 234
iou_FN: 111
iou_precision: 0.5366
iou_recall: 0.7094
iou_f1: 0.611
contain_TP: 413
contain_FP: 92
contain_FN: 123
contain_precision: 0.8178
contain_recall: 0.7705
contain_f1: 0.7935
per_image_csv: /content/drive/MyDrive/censorly-training/alcohol/contain_eval_20250917_121401/per_image_metrics.csv
boxes_detail_csv: /content/drive/MyDrive/censorly-training/alcohol/contain_eval_20250917_121401/boxes_detail.csv
viz_dir: /content/drive/MyDrive/censorly-training/alcohol/contain_eval_20250917_121401/viz

Outputs saved to: /content/drive/MyDrive/censorly-training/alcohol/contain_eval_20250917

In [ ]:
# === FP İnceleme — summary.csv Şemasına Göre Düzenlendi ===
import pandas as pd, os, shutil, math, cv2
import numpy as np
from pathlib import Path

# ----- Yollar -----
RUN_DIR = Path("/content/false_sample_test_20250917_114351")  # senin çıktı dizinin
CSV = RUN_DIR/"summary.csv"          # summary.csv (bu dizinde olmalı)
FP_DIR = RUN_DIR/"fp_review"         # inceleme klasörü
FP_IMG_DIR = FP_DIR/"images"         # orijinal img kopyaları
FP_PRED_DIR = FP_DIR/"preds"         # işaretli pred kopyaları
for d in [FP_DIR, FP_IMG_DIR, FP_PRED_DIR]:
    d.mkdir(exist_ok=True, parents=True)

# ----- CSV'yi oku (senin şema) -----
# Beklenen kolonlar: image, saved_pred, num_dets, max_conf, classes, is_false_positive
df = pd.read_csv(CSV)

# Tip güvenliği (bazı CSV'lerde bool/int karışabilir)
df["is_false_positive"] = df["is_false_positive"].astype(int)
df["num_dets"] = df["num_dets"].astype(int)
df["max_conf"] = df["max_conf"].astype(float)

# ----- FP satırları -----
fp = df[df["is_false_positive"] == 1].copy()

# Görselleri kopyala (varsa)
def safe_copy(src_path: Path, dst_path: Path):
    try:
        if src_path.exists():
            shutil.copy2(src_path, dst_path)
            return True
    except Exception as e:
        print(f"[WARN] Kopyalanamadı: {src_path} -> {dst_path.name} ({e})")
    return False

copied_img = 0
copied_pred = 0
for _, row in fp.iterrows():
    # Orijinal görsel
    img_p = Path(str(row["image"]).strip())
    if img_p.suffix.lower() in [".jpg", ".jpeg", ".png", ".bmp", ".webp"]:
        if safe_copy(img_p, FP_IMG_DIR/img_p.name):
            copied_img += 1
    # Prediction görseli (işaretli)
    pred_p = Path(str(row["saved_pred"]).strip()) if "saved_pred" in row and not pd.isna(row["saved_pred"]) else None
    if pred_p and pred_p.suffix.lower() in [".jpg", ".jpeg", ".png", ".bmp", ".webp"]:
        if safe_copy(pred_p, FP_PRED_DIR/pred_p.name):
            copied_pred += 1

print(f"[INFO] Kopyalanan orijinal img: {copied_img}")
print(f"[INFO] Kopyalanan pred görseli: {copied_pred}")

# ----- Hızlı özet: skor & tespit sayısı kümeleri -----
def bin_score(x):
    if x < 0.60: return "<0.60"
    if x < 0.75: return "0.60–0.75"
    if x < 0.90: return "0.75–0.90"
    return "≥0.90"

def bin_dets(n):
    if n == 0: return "0"
    if n == 1: return "1"
    if n <= 3: return "2–3"
    if n <= 6: return "4–6"
    return "7+"

fp["score_bin"] = fp["max_conf"].apply(bin_score)
fp["dets_bin"]  = fp["num_dets"].apply(bin_dets)

print("\n=== FP Dağılımı (score_bin x dets_bin) ===")
print(fp.groupby(["score_bin","dets_bin"]).size().sort_index())

# ----- Contact sheet (6'lı), varsa işaretli pred görsellerinden; yoksa orijinallerden -----
def make_contact_sheets(src_dir: Path, out_prefix: str, rows=2, cols=3, thumb=360):
    imgs = sorted([p for p in src_dir.glob("*") if p.suffix.lower() in [".jpg",".jpeg",".png",".bmp",".webp"]])
    if not imgs:
        print(f"[INFO] Contact için görsel yok: {src_dir}")
        return

    pages = math.ceil(len(imgs)/(rows*cols))
    for pg in range(pages):
        # beyaz tuval
        canvas = np.full((thumb*rows, thumb*cols, 3), 255, dtype=np.uint8)
        for i in range(rows*cols):
            idx = pg*(rows*cols)+i
            if idx>=len(imgs): break
            im = cv2.imread(str(imgs[idx]))
            if im is None: continue
            im = cv2.resize(im, (thumb,thumb))
            r, c = divmod(i, cols)
            canvas[r*thumb:(r+1)*thumb, c*thumb:(c+1)*thumb] = im
        out = str(RUN_DIR/f"{out_prefix}_{pg+1}.jpg")
        cv2.imwrite(out, canvas)
        print("saved", out)
# ----- (Opsiyonel) Conf eşiği süpürme — negatif sette yaklaşık FP oranı -----
# Not: summary.csv per-image 'max_conf' içerdiğinden, ~yaklaşık olarak
# conf>=t olan görüntüleri FP sayıyoruz (detay bbox bazlı değil ama pratikte iş görüyor).
import numpy as np
confs = np.linspace(0.50, 0.90, 9)
rows = []
neg_total = len(df)  # tamamı negatif set olduğundan 40
for c in confs:
    fp_at_c = (df["max_conf"] >= c).sum()
    rows.append({"conf": round(float(c),2), "approx_fp_rate": round(fp_at_c/neg_total, 4)})

print("\n=== Negatif sette yaklaşık FP oranı (max_conf eşiğine göre) ===")
for r in rows:
    print(r)


[INFO] Kopyalanan orijinal img: 15
[INFO] Kopyalanan pred görseli: 15

=== FP Dağılımı (score_bin x dets_bin) ===
score_bin  dets_bin
0.60–0.75  1           2
0.75–0.90  1           7
           2–3         3
<0.60      1           3
dtype: int64

=== Negatif sette yaklaşık FP oranı (max_conf eşiğine göre) ===
{'conf': 0.5, 'approx_fp_rate': np.float64(0.375)}
{'conf': 0.55, 'approx_fp_rate': np.float64(0.35)}
{'conf': 0.6, 'approx_fp_rate': np.float64(0.3)}
{'conf': 0.65, 'approx_fp_rate': np.float64(0.275)}
{'conf': 0.7, 'approx_fp_rate': np.float64(0.25)}
{'conf': 0.75, 'approx_fp_rate': np.float64(0.25)}
{'conf': 0.8, 'approx_fp_rate': np.float64(0.175)}
{'conf': 0.85, 'approx_fp_rate': np.float64(0.15)}
{'conf': 0.9, 'approx_fp_rate': np.float64(0.0)}


In [ ]:
from ultralytics import YOLO
model = YOLO("/content/drive/MyDrive/censorly-training/alcohol/finetune_with_all_neg/weights/best.pt")

model.train(
    data="/content/drive/MyDrive/censorly-training/alcohol/data_with_all_neg.yaml",
    epochs=12, imgsz=640, batch=-1,
    lr0=0.002, lrf=0.01, cos_lr=True, patience=5,
    # daha sakin augment
    hsv_h=0.010, hsv_s=0.4, hsv_v=0.25,
    mosaic=0.3, erasing=0.0, fliplr=0.5,
    perspective=0.0003, mixup=0.0, copy_paste=0.0,
    # loss ağırlıkları
    box=7.5, dfl=1.5, cls=0.5,
    deterministic=True,
)


Ultralytics 8.3.201 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=-1, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/content/drive/MyDrive/censorly-training/alcohol/data_with_all_neg.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=12, erasing=0.0, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.01, hsv_s=0.4, hsv_v=0.25, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.002, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=/content/drive/MyDrive/censorly-training/alcohol/finetune_with_all_neg/weights/best.pt, momentum=0.937, mosaic=0.3, multi_scale=False, name=train2, n

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7df3fd9071a0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

In [ ]:
import shutil
from pathlib import Path

# kaynak ve hedef
src = Path("/content/runs/detect/train2")
dst = Path("/content/drive/MyDrive/censorly-training/alcohol/finetune_with_all_neg2")

# varsa önce silmek için (opsiyonel)
if dst.exists():
    shutil.rmtree(dst)

# klasörü komple kopyala
shutil.copytree(src, dst)

print(f"[OK] Fine-tune sonuçları Drive'a kopyalandı → {dst}")


[OK] Fine-tune sonuçları Drive'a kopyalandı → /content/drive/MyDrive/censorly-training/alcohol/finetune_with_all_neg2


In [ ]:
# Bu hücre, False tarafında eksik boş .txt varsa üretir. Varsa atla.
FALSE_LBL = Path(f"{ROOT}/Alcohol-False/Alcohol-False/labels")
created = 0
for split in ["train","validation","test"]:
    img_dir = FALSE_IMG / split
    lbl_dir = FALSE_LBL / split
    if not img_dir.exists():
        continue
    lbl_dir.mkdir(parents=True, exist_ok=True)
    for f in img_dir.rglob("*"):
        if f.is_file() and f.suffix.lower() in EXTS:
            t = lbl_dir / (f.stem + ".txt")
            if not t.exists():
                t.write_text("")  # negatif -> boş label
                created += 1
print("Oluşturulan boş label sayısı:", created)


Oluşturulan boş label sayısı: 0


In [ ]:
from pathlib import Path

DRIVE = "/content/drive/MyDrive"
ROOT  = f"{DRIVE}/censorly-dataset/Alcohol"

TRUE2_IMG = Path(f"{ROOT}/Alcohol-True/Edited-Alcohol-2/images")
TRUE2_LBL = Path(f"{ROOT}/Alcohol-True/Edited-Alcohol-2/labels")

FALSE_IMG = Path(f"{ROOT}/Alcohol-False/Alcohol-False/images")
FALSE_LBL = Path(f"{ROOT}/Alcohol-False/Alcohol-False/labels")

EXTS = {".jpg",".jpeg",".png",".bmp",".webp"}


In [ ]:
def list_images(base: Path, split: str):
    p = base / split
    if not p.exists(): return []
    return [str(f.resolve()) for f in sorted(p.rglob("*"))
            if f.is_file() and f.suffix.lower() in EXTS]

splits = {}
for split in ["train","validation","test"]:
    imgs_true  = list_images(TRUE2_IMG, split)
    imgs_false = list_images(FALSE_IMG, split)
    splits[split] = imgs_true + imgs_false
    print(f"{split}: true={len(imgs_true)} false={len(imgs_false)} total={len(splits[split])}")

# TXT dosyalarına yaz
TXT_DIR = Path("/content/alcohol_lists"); TXT_DIR.mkdir(parents=True, exist_ok=True)
paths = {}
for split in ["train","validation","test"]:
    p = TXT_DIR / f"{split}.txt"
    p.write_text("\n".join(splits[split]))
    paths[split] = str(p)
    print("wrote:", p)


train: true=1847 false=450 total=2297
validation: true=231 false=123 total=354
test: true=226 false=65 total=291
wrote: /content/alcohol_lists/train.txt
wrote: /content/alcohol_lists/validation.txt
wrote: /content/alcohol_lists/test.txt


In [ ]:
DATA_YAML = "/content/alcohol_multi_plus_false_lists.yaml"

names_block = [
 'alcohol rack', 'beer bottle', 'beer can', 'beer glass',
 'champagne bottle', 'champagne glass', 'cocktail',
 'liquor bottle', 'liquor glass', 'vodka bottle', 'vodka glass',
 'whiskey bottle', 'whiskey glass', 'white wine glass',
 'wine bottle', 'wine glass'
]

data_yaml = f"""# Multiclass positives (Edited-Alcohol-2) + negatives (Alcohol-False) via TXT lists
train: {paths['train']}
val:   {paths['validation']}
test:  {paths['test']}

nc: 16
names: {names_block}
"""

with open(DATA_YAML, "w") as f:
    f.write(data_yaml)

print("Wrote:", DATA_YAML)


Wrote: /content/alcohol_multi_plus_false_lists.yaml


In [ ]:
!pip install -U ultralytics
from ultralytics import YOLO

# Eski alkol ağırlıklarını kullanmıyoruz (bias taşımamak için), COCO ön-eğitimli backbone ile başlıyoruz
model = YOLO('yolov8m.pt')   # hız için 'yolov8s.pt' de tercih edebilirsin

model.train(
    data=DATA_YAML,
    epochs=80,
    imgsz=1280,              # küçük kadeh/etiket için faydalı
    batch=16,
    device=0,

    # FP'yi bastırmaya yönelik:
    lr0=0.01, lrf=0.01, weight_decay=0.0005,
    optimizer='SGD',
    cos_lr=True,             # daha stabil iniş

    # Augment (abartmadan çeşitlilik):
    hsv_h=0.015, hsv_s=0.7, hsv_v=0.7,
    degrees=7.0, translate=0.05, scale=0.5, shear=1.0,
    perspective=0.0005, flipud=0.0, fliplr=0.5,
    mosaic=0.55, close_mosaic=12, mixup=0.0, copy_paste=0.05,

    project="runs-alcohol", name="scratch_multi_hardneg_v1",
    patience=30
)


Ultralytics 8.3.204 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=12, cls=0.5, compile=False, conf=None, copy_paste=0.05, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/content/alcohol_multi_plus_false_lists.yaml, degrees=7.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=80, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.7, imgsz=1280, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8m.pt, momentum=0.937, mosaic=0.55, multi_scale=False, name=scratch_multi_hardneg_v1, nbs=64, nms=False, opset=None, optimize=False, optimizer=SGD, overlap_mask=True, pat

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([ 0,  1,  2,  3,  4,  5,  6,  7,  8, 10, 11, 12, 13, 14, 15])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x78e731bfb7d0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,   

In [ ]:
from pathlib import Path

DRIVE = "/content/drive/MyDrive"
ROOT  = f"{DRIVE}/censorly-dataset/Alcohol"

TRUE3_IMG = Path(f"{ROOT}/Alcohol-True/Edited-Alcohol-3/images")  # tek sınıf
FALSE_IMG = Path(f"{ROOT}/Alcohol-False/Alcohol-False/images")    # negatif

EXTS = {".jpg",".jpeg",".png",".bmp",".webp"}

def list_images(base: Path, split: str):
    p = base / split
    if not p.exists(): return []
    return [str(f.resolve()) for f in sorted(p.rglob("*")) if f.suffix.lower() in EXTS]

splits = {}
for split in ["train","validation","test"]:
    splits[split] = list_images(TRUE3_IMG, split) + list_images(FALSE_IMG, split)

from pathlib import Path
TXT_DIR = Path("/content/alcohol_bin_lists"); TXT_DIR.mkdir(parents=True, exist_ok=True)
paths = {}
for s in ["train","validation","test"]:
    p = TXT_DIR/f"{s}.txt"; p.write_text("\n".join(splits[s])); paths[s]=str(p)
print(paths)


{'train': '/content/alcohol_bin_lists/train.txt', 'validation': '/content/alcohol_bin_lists/validation.txt', 'test': '/content/alcohol_bin_lists/test.txt'}


In [ ]:
from pathlib import Path
DRIVE="/content/drive/MyDrive"; ROOT=f"{DRIVE}/censorly-dataset/Alcohol"
TRUE_IMG=Path(f"{ROOT}/Alcohol-True/Edited-Alcohol-2/images")
FALSE_IMG=Path(f"{ROOT}/Alcohol-False/Alcohol-False/images")
EXTS={".jpg",".jpeg",".png",".bmp",".webp"}

def collect(p):
    return [str(f.resolve()) for f in sorted(p.rglob("*")) if f.suffix.lower() in EXTS]

ratio_neg = 1.5  # negatifleri ~%50 daha fazla göster (FP’yi kırar)
paths={}
TXT_DIR=Path("/content/alcohol_multi_lists"); TXT_DIR.mkdir(parents=True, exist_ok=True)
for split in ["train","validation","test"]:
    pos = collect(TRUE_IMG/split)
    neg = collect(FALSE_IMG/split)
    overs = (neg * int(ratio_neg)) + neg[: int(len(neg)*(ratio_neg%1))]
    allp = pos + overs if split=="train" else (pos + neg)
    p = TXT_DIR/f"{split}.txt"; p.write_text("\n".join(allp)); paths[split]=str(p)
    print(split, "pos",len(pos),"neg",len(neg),"train_used",len(allp))


train pos 1847 neg 450 train_used 2522
validation pos 231 neg 123 train_used 354
test pos 226 neg 65 train_used 291


In [ ]:
DATA_YAML="/content/alcohol_multi_plus_false.yaml"
names_block=[
 'alcohol rack','beer bottle','beer can','beer glass',
 'champagne bottle','champagne glass','cocktail',
 'liquor bottle','liquor glass','vodka bottle','vodka glass',
 'whiskey bottle','whiskey glass','white wine glass',
 'wine bottle','wine glass'
]
open(DATA_YAML,"w").write(f"""
train: {paths['train']}
val:   {paths['validation']}
test:  {paths['test']}
nc: 16
names: {names_block}
""")
print(DATA_YAML)


/content/alcohol_multi_plus_false.yaml


In [ ]:

from ultralytics import YOLO

model = YOLO("yolov8m.pt") 

model.train(
    data=DATA_YAML,
    epochs=100,
    imgsz=1280,
    batch=16,
    device=0,

    # FP'yi düşürmeye odak
    lr0=0.01, lrf=0.01, weight_decay=0.0005,
    optimizer="SGD", cos_lr=True,

    # Augment
    mosaic=0.5, mixup=0.0, copy_paste=0.0,
    hsv_h=0.015, hsv_s=0.6, hsv_v=0.6,
    degrees=7.0, translate=0.05, scale=0.5, shear=1.0,
    perspective=0.0005, flipud=0.0, fliplr=0.5,

    close_mosaic=12,               # son 12 epoch mozaik kapalı 
    patience=40,

    project="runs-alcohol", name="scratch_multi_hardneg_v2"
)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 60.9 MB/s eta 0:00:00
  Attempting uninstall: ultralytics
    Found existing installation: ultralytics 8.3.199
    Uninstalling ultralytics-8.3.199:
      Successfully uninstalled ultralytics-8.3.199
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.3.204 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=12, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/content/alcohol_multi_plus_false.yaml, degrees=7.0, deterministic

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([ 0,  1,  2,  3,  4,  5,  6,  7,  8, 10, 11, 12, 13, 14, 15])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7d1ada5143b0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,   